# Tomo extended

## Import modules

In [2]:
# Basic stuff
import sys
sys.path.append("/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/lib/python/")
import genericIO
import SepVector
import Hypercube
import Acoustic_iso_double
import numpy as np
import time

# Plotting library
import matplotlib.pyplot as plt
import sepPlot

## Wavelet (sources)

In [3]:
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/waveletMain.py timeDelay=1.0 f1=2 f2=5 f3=8 f4=15 par=parTomoExt.p type=ali wavelet=waveletTomoExt.H

## Velocity models

In [4]:
# True velocity model
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/padFileGpuMain zPad=100 xPad=100 model=velocityMarmousi.H data=velocityMarmousi.pad.H

# Background velocity model
!Smooth rect1=10 rect2=10 < velocityMarmousi.H > velocityMarmousiSmooth.H
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/padFileGpuMain zPad=100 xPad=100 model=velocityMarmousiSmooth.H data=velocityMarmousiSmooth.pad.H

 
------------------------ Model padding program --------------------
Original nz = 350 [samples]
Original nx = 1700 [samples]
 
zPadMinus = 100 [samples]
zPadPlus = 110 [samples]
xPadMinus = 100 [samples]
xPadPlus = 104 [samples]
 
blockSize = 16 [samples]
FAT = 5 [samples]
 
New nz = 570 [samples including padding and FAT]
New nx = 1914 [samples including padding and FAT]
-------------------------------------------------------------------
 
 
------------------------ Model padding program --------------------
Original nz = 350 [samples]
Original nx = 1700 [samples]
 
zPadMinus = 100 [samples]
zPadPlus = 110 [samples]
xPadMinus = 100 [samples]
xPadPlus = 104 [samples]
 
blockSize = 16 [samples]
FAT = 5 [samples]
 
New nz = 570 [samples including padding and FAT]
New nx = 1914 [samples including padding and FAT]
-------------------------------------------------------------------
 


## Extended reflectivity

In [5]:
# Create an extended reflectivity
!Spike n1=350 n2=1700 d1=1.0 d2=1.0 n3=1 d3=1.0 velback=0.0 nsp=1 mag=0.01 k1=250 k2=850 > junk.H
!Pad beg3=5 end3=15 extend=0 < junk.H > tomoOffsetReflectivity.H
!echo "d1=0.01 d2=0.01 d3=0.01 o1=0.0 o2=0.0 o3=-0.1" >> tomoOffsetReflectivity.H

# Pad extended reflectivity
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/padFileGpuMain zPad=100 xPad=100 model=tomoOffsetReflectivity.H data=tomoOffsetReflectivity.pad.H

 
------------------------ Model padding program --------------------
Original nz = 350 [samples]
Original nx = 1700 [samples]
 
zPadMinus = 100 [samples]
zPadPlus = 110 [samples]
xPadMinus = 100 [samples]
xPadPlus = 104 [samples]
 
blockSize = 16 [samples]
FAT = 5 [samples]
 
New nz = 570 [samples including padding and FAT]
New nx = 1914 [samples including padding and FAT]
-------------------------------------------------------------------
 


## Model (background perturbation)

In [5]:
# Background perturbation
!Spike n1=350 n2=1700 d1=1.0 d2=1.0 n3=1 d3=1.0 velback=0.0 nsp=1 mag=0.01 k1=250 k2=850 > tomoModel.H

# Pad reflectivity model
!/net/server/homes/sep/gbarnier/code/gpu/acousticIsoOp/test/bin/padFileGpuMain zPad=100 xPad=100 model=tomoModel.H data=tomoModel.pad.H

 
------------------------ Model padding program --------------------
Original nz = 350 [samples]
Original nx = 1700 [samples]
 
zPadMinus = 100 [samples]
zPadPlus = 110 [samples]
xPadMinus = 100 [samples]
xPadPlus = 104 [samples]
 
blockSize = 16 [samples]
FAT = 5 [samples]
 
New nz = 570 [samples including padding and FAT]
New nx = 1914 [samples including padding and FAT]
-------------------------------------------------------------------
 


## Initialize tomo extended operator

In [6]:
args=["dummy arg","vel=velocityMarmousiSmooth.pad.H","sources=waveletTomoExt.H","par=parTomoExt.p","model=tomoModel.pad.H","reflectivity=tomoOffsetReflectivity.pad.H","extension=offset","nExt=21"]
modelDouble,dataDouble,velDouble,parObject,sourcesVector,sourcesSignalsVector,receiversVector,reflectivitydouble=Acoustic_iso_double.tomoExtOpInitDouble(args)

# Printing parameter file for reference
!cat parTomoExt.p

nts=1000
dts=0.008
sub=8
nz=570
nx=1914
zPadMinus=100
zPadPlus=110
xPadMinus=100
xPadPlus=104
dz=0.01
dx=0.01
fMax=20
zSource=2
xSource=850
nShot=1
spacingShots=10
depthReceiver=2
nReceiver=1700
dReceiver=1
oReceiver=1
blockSize=16
fat=5
nGpu=8


## Create tomo object

In [7]:
tomoExtOp=Acoustic_iso_double.tomoExtShotsGpu(modelDouble,dataDouble,velDouble,parObject,sourcesVector,sourcesSignalsVector,receiversVector,reflectivitydouble)

## Read model (background perturbation)

In [8]:
modelFile=parObject.getString("model")
modelFloat=genericIO.defaultIO.getVector(modelFile,ndims=2)
modelDouble=SepVector.getSepVector(modelFloat.getHyper(),storage="dataDouble")
modelDMat=modelDouble.getNdArray()
modelSMat=modelFloat.getNdArray()
modelDMat[:]=modelSMat

## Tomo extended forward

In [ ]:
tomoExtOp.forward(False,modelDouble,dataDouble)